In [ ]:
! [ -e /content ] && pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz

In [ ]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8
validationLookback = 7

In [ ]:
path = Path.cwd()
Path.BASE_PATH = path
dsets_path = path.parent.parent/'fastai/datasets'

In [ ]:
df = pd.read_csv(dsets_path/'osrs_trades.csv', low_memory=False)
df.columns

In [ ]:
dep_var = 'gst'

In [ ]:
make_date(df, 'timestamp')
df = add_datepart(df, 'timestamp')
df = df.drop('id', axis=1)
df.columns

In [ ]:
procs = [Categorify, FillMissing]
maxDay = df['timestampDayofyear'].max()
maxDay

In [ ]:
minCond = (df.timestampDayofyear>120) # 117 is when I fixed date recording on completed_trades
maxCond = (df.timestampDayofyear<=(maxDay - validationLookback))
train_idx = np.where((maxCond) & (minCond))[0]
valid_idx = np.where(~maxCond)[0]

splits = (list(train_idx),list(valid_idx))

In [ ]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [ ]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [ ]:
save_pickle(path/'preprocessed_train_valid.pkl', to)

In [ ]:
to = load_pickle(path/'preprocessed_train_valid.pkl')

In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [ ]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [ ]:
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(to.train.xs, to.train.y);
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)